In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, r2_score

In [2]:
data = pd.read_csv('train.csv', index_col = 'Id')
data_0 = data.copy()
data.corr()['mean_exam_points']


age                   -0.007646
years_of_experience    0.205417
lesson_price           0.721179
qualification          0.755963
physics                0.187726
chemistry              0.017825
biology                0.023022
english                0.013174
geography              0.014401
history               -0.000113
mean_exam_points       1.000000
Name: mean_exam_points, dtype: float64

In [3]:
def min_max(x):
    res = x.copy()
    for i in range(x.shape[1]):
        res.iloc[:,i]= (res.iloc[:,i] - res.iloc[:,i].min()) / (res.iloc[:,i].max() - res.iloc[:,i].min())
    return res

In [4]:
lessons = ['history','english', 'physics','chemistry','geography','biology']
data.iloc[:,:-1] = min_max(data.iloc[:,:-1])
data.insert(3,'les',(sum([data[i]*data.corr()['mean_exam_points'][i] for i in lessons])))
data.drop(columns =['age', *lessons], inplace = True)
data.corr()['mean_exam_points']

years_of_experience    0.205417
lesson_price           0.721179
les                    0.190075
qualification          0.755963
mean_exam_points       1.000000
Name: mean_exam_points, dtype: float64

In [5]:
data_X = data.iloc[:,:-1].to_numpy()
data_y = data.iloc[:,-1].to_numpy()


In [6]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [7]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        prediction = np.mean(self.labels)
        return prediction

In [8]:
def criterian(labels):
    mse =  np.std(labels)
    return mse

In [9]:
def quality(left_labels, right_labels, current_criterian):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_criterian - p * criterian(left_labels) - (1 - p) *criterian(right_labels)

In [10]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [11]:
# Нахождение наилучшего разбиения


def find_best_split(data, labels, depth):
    
    np.random.seed(3664452660)
    
    #  обозначим минимальное количество объектов в узле
    
    min_leaf =15

    current_criterian = criterian(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    n_f = np.arange(n_features)
#     n_features_r=np.random.choice(n_f,size=9,replace=False)

    
    for index in range(n_features):
        t_values =list(set([row[index] for row in data]))
        t_values_r= np.random.choice(t_values,size=int(len(t_values)*0.1), replace=False)
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criterian)
#             print(current_quality)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index
#                 print(best_quality)

    return best_quality, best_t, best_index

In [12]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, depth=1, depth_max = 6,  d=0):

    quality, t, index = find_best_split(data, labels, depth)
#     print(quality)


    #  Базовый случай - прекращаем рекурсию, когда прирост качества меньше d или дерево достигло максимальной глубины
    #  или количество потенциальных листьев достигло максимума(что при данном алгоритме идентично глубине) или все элементы одного класса
    if quality <= d or depth>=depth_max :
#         print(d,quality, depth, leaves)
        
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    depth +=1
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth, depth_max, d)
    false_branch = build_tree(false_data, false_labels, depth, depth_max, d)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [13]:
# Проход объекта по дереву для его классификации

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [14]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [15]:
def bias(y, z):
    return (y - z)

In [16]:
def gb_fit(n_trees, X_train,X_test, y_train, y_test, eta):
    
# Деревья будем записывать в список

    trees = []
    
    for i in range(n_trees):
        
        # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
        # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
   
        if len(trees) == 0:
            tree = build_tree(X_train,y_train, depth_max=10, d=0.1)
            trees.append(tree)
            targets_train = np.array(predict(X_train, tree))
            targets_test=np.array(predict(X_test, tree))
#             print(r2_score(y_train, targets_train))
#             print(r2_(y_test, targets_test))
            
        else:
            tree=build_tree(X_train, (eta/(1+i*0.0001))*bias(y_train,targets_train),depth_max=6, d=0.001)           
            trees.append(tree)
            targets_train += np.array(predict(X_train, tree))
            targets_test+=np.array(predict(X_test, tree))
#             print(r2_score(y_train, targets_train))
#             print(r2_(y_test, targets_test))

    return trees, targets_train,  targets_test

In [17]:
def r2_(y,X):
    r2_score = 1 - ((np.array(X)-np.array(y))**2).sum()/((np.array(y)-np.mean(y))**2).sum()
    return r2_score

In [18]:
# np.random.seed(101)

In [19]:
# Разобьем выборку на обучающую и тестовую

train_data, test_data, train_labels, test_labels = train_test_split(data_X, 
                                                                    data_y, 
                                                                    test_size = 0.3,random_state=100)


In [20]:
# Число деревьев в ансамбле
n_trees = 20


# Шаг
eta =0.5


In [21]:
trees, tagets_train, tagets_test = gb_fit(n_trees,train_data, test_data, train_labels, test_labels, eta)

In [22]:
r2_score(train_labels, tagets_train), r2_score(test_labels, tagets_test)

(0.7859561716113276, 0.7899151486139392)

In [23]:
data_test = pd.read_csv('test.csv', index_col = 'Id')
lessons = ['history','english', 'physics','chemistry','geography','biology']
data_test = min_max(data_test)
data_test.insert(3,'les',(sum([data_test[i]*data_0.corr()['mean_exam_points'][i] for i in lessons])))
data_test.drop(columns =['age',*lessons], inplace = True)
data_test_x=data_test.to_numpy()

In [24]:
trees, tagets_train, tagets_test = gb_fit(n_trees,data_X, data_test_x, data_y, test_labels, eta)

In [25]:

pd_pred= pd.DataFrame({'Id':data_test.index, 'mean_exam_points': tagets_test}, )
pd_pred.to_csv("submission.csv", index=False)